In [ ]:
import openpyxl
from bs4 import BeautifulSoup
import os
import requests

def data_scrapper():

    url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW'
    # open the navigator with a mask
    browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.125 Safari/537.36 Edg/108.0.1462.54"}
    # Get site with url
    page = requests.get(url, headers=browsers)
    
    # Rescuin HTML from BeautifulSoup
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Rescuing Header
    headers = soup.find_all('th')
    
    # Inserting data in a list
    header_list = []
    for header in headers:
        header_list.append(header.text)
    
    
    # Rescuing Data
    movie_ranks = soup.find_all('td', {'class': 'mojo-field-type-rank'})
    movie_titles = soup.find_all('td', {'class': 'mojo-field-type-title'})
    movie_lg = soup.find_all('td', {'class': 'mojo-field-type-money'})
    movie_year = soup.find_all('td', {'class': 'mojo-field-type-year'})

    rank_list = []
    title_list = []
    money_list = []
    year_list = []

    # Adding data to lists
    for rank in movie_ranks:
        rank_list.append(rank.text)
    
    for title in movie_titles:
        title_list.append(title.text)

    for money in movie_lg:
        item = money.text
        item = item.replace('$', '')
        item = item.replace(',', '')
        item = float(item)
        money_list.append(item)
    
    for year in movie_year:
        year_list.append(year.text)


    os.makedirs("C:/Movie/movie_excel")
    # Creating and open Excel file
    path = "C:/Movie/movie_excel/movie.xlsx"
    wb = openpyxl.Workbook()
    sheet = wb.active

    # Defining the Header Design
    header_style = openpyxl.styles.NamedStyle(name='header_style')
    header_style.font = openpyxl.styles.Font(name='Times New Roman', size= 14, bold= True, color = 'FFFFFFFF')
    header_style.fill = openpyxl.styles.PatternFill(fill_type= 'solid', start_color= '00000000')
    header_style.alignment = openpyxl.styles.Alignment(horizontal= 'center', vertical= 'center')

    # Defining the Data Design
    data_style = openpyxl.styles.NamedStyle(name='data_style')
    data_style.font = openpyxl.styles.Font(name='Times New Roman', size= 12)
    data_style.alignment = openpyxl.styles.Alignment(horizontal= 'center', vertical= 'center')

    # Putting the data in the respective columns (header, rank, title, money, Year) in the Excel Sheet
    for valor in range(0,len(header_list)):
        cell = sheet.cell(column = valor + 1, row = 1)
        cell.value = header_list[valor]
        cell.style = header_style
    
    for valor in range(0,len(rank_list)):
        cell = sheet.cell(row = valor + 2, column = 1)
        cell.value = rank_list[valor]
        cell.style = data_style

    for valor in range(0,len(title_list)):
        cell = sheet.cell(row = valor + 2, column = 2)
        cell.value = title_list[valor]
        cell.style = data_style

    for valor in range(0,len(money_list)):
        cell = sheet.cell(row = valor + 2, column = 3)
        cell.value = money_list[valor]
        cell.style = data_style

    for valor in range(0,len(year_list)):
        cell = sheet.cell(row = valor + 2, column = 4)
        cell.value = year_list[valor]
        cell.style = data_style

    # Apply filters in the columns
    filters = sheet.auto_filter
    filters.ref = 'A1:D201'

    # Creating new Sheet for chart
    chart_sheet = wb.create_sheet("chart_table")

    checked_year = []
    lg_total_list = []
    treated_dt_header = [ 'Year', 'Lifetime Gross']

    # Separate the Lifetime Grosses values by year
    for i in range(0, len(year_list)):
        lg_total = 0
        for x in range(0, len(year_list)):
            if year_list[i] == year_list[x] and year_list[i] not in checked_year:
                lg_total += money_list[x]
        if year_list[i] not in checked_year:
            checked_year.append(year_list[i])
            lg_total_list.append(lg_total)
    display(checked_year, lg_total_list, treated_dt_header)
    
    # Putting the data in the respective columns (Year, Lifetime Gross) in the Chart Sheet
    for valor in range(0,len(treated_dt_header)):
        cell = chart_sheet.cell(column = valor + 1, row = 1)
        cell.value = treated_dt_header[valor]
        cell.style = header_style
    
    for valor in range(0,len(lg_total_list)):
        cell = chart_sheet.cell(row = valor + 2, column = 2)
        cell.value = lg_total_list[valor]
        cell.style = data_style

    for valor in range(0,len(checked_year)):
        cell = chart_sheet.cell(row = valor + 2, column = 1)
        cell.value = checked_year[valor]
        cell.style = data_style
    
    # Apply filters in the columns
    filters = chart_sheet.auto_filter
    filters.ref = 'A1:B30'

    # Create a bar chart
    chart1 = openpyxl.chart.BarChart()
    chart1.type = 'col'
    chart1.style = 10
    chart1.title = 'Best year for Cinema'
    chart1.y_axis.title = 'Lifetime Gross'
    chart1.x_axis.title = 'Year'
    # Put the data in the chart
    data = openpyxl.chart.Reference(chart_sheet, min_col = 2, min_row = 2, max_row = 30, max_col = 2)
    cats = openpyxl.chart.Reference(chart_sheet, min_col = 1, min_row = 2, max_row = 30, max_col = 1)
    chart1.add_data(data, titles_from_data = True)
    chart1.set_categories(cats)
    chart1.shape = 4
    chart_sheet.add_chart(chart1, "D1")


    wb.active = chart_sheet
    wb.save(path)
    os.startfile(path)
    





data_scrapper()